import pandas as pd
import os
import re

rainfall_data = pd.DataFrame()
for file in os.listdir("lawas"):
    if re.search(r"rainfall-data\d+\.csv", file):
        df = pd.read_csv(f"lawas/{file}")
        rainfall_data = pd.concat([rainfall_data, df], ignore_index=True)

rainfall_data.to_csv("lawas/rainfall-data-combine.csv")

# Feature Engineering

In [ ]:
import pandas as pd

df = pd.read_csv("lawas/rainfall-data-combine.csv",usecols=["DateTime", "Rainfall (mm)"])
df["DateTime"] = pd.to_datetime(df["DateTime"],format="%Y%m%d %H%M")



# LSTM Model Univariate